In [1]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
# %tensorflow_version 2.x
# import tensorflow as tf
# tf.test.gpu_device_name()
# from scipy import integrate
# import os
# import numpy as np
# from tensorflow import keras
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K


from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr.clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot



import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from attention import Attention

C:\Users\moshi\AppData\Local\Temp/ipykernel_12540/628588012.py:43: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
data_train = pd.read_csv('IR_Train.csv',encoding= 'unicode_escape') 
data_test = pd.read_csv('IR_Test.csv',encoding= 'unicode_escape')
data_val= pd.read_csv('IR_Val.csv',encoding= 'unicode_escape')

X_train = data_train.Utterance.tolist()
X_test = data_test.Utterance.tolist()
X_val = data_train.Utterance.tolist()

y_train = data_train.Intent.tolist()
y_test = data_test.Intent.tolist()
Y_val = data_test.Intent.tolist()



data = data_train.append(data_test, ignore_index=True)
temp= data.append(data_val, ignore_index=True)


print('size of training set: %s' % (len(data_train['Utterance'])))
print('size of testing set: %s' % (len(data_test['Utterance'])))
print('size of validation set: %s' % (len(data_val['Utterance'])))
print(temp.Intent.value_counts())

temp.head(10)

size of training set: 15000
size of testing set: 4500
size of validation set: 3000
translate          150
order_status       150
goodbye            150
account_blocked    150
what_song          150
                  ... 
reminder           150
change_speed       150
tire_pressure      150
no                 150
card_declined      150
Name: Intent, Length: 150, dtype: int64


,Utterance,Intent
0,what expression would i use to say i love you ...,translate
1,can you tell me how to say 'i do not speak muc...,translate
2,"what is the equivalent of, 'life is good' in f...",translate
3,"tell me how to say, 'it is a beautiful morning...",translate
4,"if i were mongolian, how would i say that i am...",translate
5,how do i say 'hotel' in finnish,translate
6,"i need you to translate the sentence, 'we will...",translate
7,please tell me how to ask for a taxi in french,translate
8,"can you tell me how i would say, 'more bread p...",translate
9,what is the correct way to say 'i am a visitor...,translate


In [4]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
temp["Intent_label"] = lb_make.fit_transform(temp["Intent"])
temp[["Intent", "Intent_label"]].head(11)

,Intent,Intent_label
0,translate,131
1,translate,131
2,translate,131
3,translate,131
4,translate,131
5,translate,131
6,translate,131
7,translate,131
8,translate,131
9,translate,131


In [8]:
train, test = train_test_split(temp, test_size=0.2, stratify = temp['Intent_label'], random_state = 42)
num_classes = 150
embed_num_dims = 300
max_seq_len = 50

x_train = train['Utterance']
x_test = test['Utterance']

y_train = train['Utterance']
y_test = test['Utterance']

texts_train = x_train
texts_test = x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['Utterance'])

sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index

vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len, padding='pre' )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len,  padding='pre')

print(X_train_pad)



y_train = train['Intent_label']
y_test = test['Intent_label']



y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)






Number of unique words: 5822
[[  0   0   0 ... 167 369  19]
 [  0   0   0 ...  16 799 231]
 [  0   0   0 ...   1  73  18]
 ...
 [  0   0   0 ...  89 646  35]
 [  0   0   0 ...   1 305 113]
 [  0   0   0 ...   3  71  26]]


In [9]:
def create_embedding_matrix(word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    with open('cc.en.300.vec',encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
    return embedding_matrix


embedd_matrix = create_embedding_matrix(index_of_words, embed_num_dims)
print(embedd_matrix.shape)

(5823, 300)



# Random Search


In [ ]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =32, max_value = 128, step = 16)
  fake_val = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =64, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =64, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)



  seq_input = keras.layers.Input(shape=(max_seq_len,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = max_seq_len,
                          weights = [embedd_matrix])(seq_input)

  cnn = keras.layers.Conv1D(cnn_1_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)

  attention_vec = keras.layers.TimeDistributed(keras.layers.Dense(unit_attention))(cnn)
  attention_vec = keras.layers.Reshape((48,unit_attention))(attention_vec)
  attention_vec = keras.layers.Activation('relu', name = 'cnn_attention_vec')(attention_vec)
  attention_output = keras.layers.Dot(axes = 1)([cnn, attention_vec])


  lstm = keras.layers.Bidirectional(keras.layers.LSTM(lstm_unit, recurrent_regularizer=regularizers.l2(1e-4),
                                                      return_sequences=True,kernel_regularizer=regularizers.l2(1e-4),
                                                      bias_regularizer=regularizers.l2(1e-2),
                                                      activity_regularizer=regularizers.l2(1e-4),input_shape =(48,)))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  cnn_2 = keras.layers.Conv1D(cnn_2_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(num_classes, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  return model


stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


clr_step_size = int((len(X_train_pad)/64))
base_lr = 1e-3
max_lr = 6e-3
mode = 'exp_range'


clr = CyclicLR(base_lr = base_lr, max_lr = max_lr, step_size = clr_step_size, mode = mode)



tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 50,
    executions_per_trial = 1,
    directory = LOG_DIR
    )
  
tuner.search(x=X_train_pad,y = y_train,epochs = 20, batch_size = 128,callbacks = [stop,clr], 
             validation_data = (X_test_pad,y_test))


tuner.results_summary()

Trial 40 Complete [00h 17m 18s]
val_accuracy: 0.8640000224113464

Best val_accuracy So Far: 0.9200000166893005
Total elapsed time: 14h 05m 57s

Search: Running Trial #41

Hyperparameter    |Value             |Best Value So Far 
attention_unit    |80                |48                
cnn_1_unit        |96                |48                
cnn_1_dropout     |0.3               |0.2               
lstm_unit         |224               |160               
lstm_dropout      |0.2               |0.1               
cnn_2_unit        |160               |224               
cnn_2_dropout     |0.3               |0.5               

Ya it comes here
Epoch 1/20
141/141 [==============================] - 148s 1s/step - loss: 10.8019 - accuracy: 0.0626 - val_loss: 7.4800 - val_accuracy: 0.0180
Epoch 2/20
141/141 [==============================] - 139s 983ms/step - loss: 5.7749 - accuracy: 0.2362 - val_loss: 4.9234 - val_accuracy: 0.1962
Epoch 3/20
141/141 [==============================] - 139s 986ms/